In [1]:
!pip install catboost

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
!pip install catboost
from catboost import CatBoostRegressor
#from sklearn.multioutput import MultiOutputRegressor as mor
from xgboost import XGBRegressor
!pip install lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import RandomForestRegressor as rf 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor as ann
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import joblib

In [3]:
data=pd.read_excel("corrected_filenamee_daylight.xlsx")

x1=data.iloc [:, :10]
y=data.iloc [:, 11:]

t = tts(x1, y, random_state=42, test_size=0.2)

In [4]:
data.columns

Index([' SP-Soth-Dis', ' SP-East-Dis', ' SP-North-Dis', ' SP-West-Dis',
       'Total-Sky-cover', 'Altitude', 'Azimuth', 'AP-Length', 'AP-Width',
       'SP-Ap-Dis', 'Et', 'Ev'],
      dtype='object')

In [5]:
x1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44908 entries, 0 to 44907
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0    SP-Soth-Dis     44908 non-null  float64
 1    SP-East-Dis     44908 non-null  float64
 2    SP-North-Dis    44908 non-null  float64
 3    SP-West-Dis     44908 non-null  float64
 4   Total-Sky-cover  44908 non-null  int64  
 5   Altitude         44908 non-null  float64
 6   Azimuth          44908 non-null  float64
 7   AP-Length        44908 non-null  float64
 8   AP-Width         44908 non-null  float64
 9   SP-Ap-Dis        44908 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 3.4 MB


In [5]:
data1= data.copy()
data2= data1.query("300 <Et< 1500")
data3= data2.query("Ev<4000") # new dataset

In [10]:
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = tts(x1, y, test_size=0.2, random_state=42)


In [13]:

def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse


In [4]:
#lgb

import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Assuming you have imported your data as 'data3'

# Separate features and target variable
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.2, random_state=42)

# LightGBM
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'num_leaves': [31, 50, 100],
}

# Evaluate models
def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

lgb_grid = GridSearchCV(lgb_model, lgb_params, scoring='neg_mean_squared_error', cv=3)
lgb_grid.fit(x_train, y_train.values.ravel())

# Get the best parameters and model
best_lgb_params = lgb_grid.best_params_
best_lgb_model = lgb_grid.best_estimator_
print("Best Parameters for light g boost:",best_lgb_params )

# Evaluate LightGBM
lgb_r2, lgb_mae, lgb_mse = evaluate_model(best_lgb_model, x_test, y_test)
print(f"LightGBM - R2: {lgb_r2}, MAE: {lgb_mae}, MSE: {lgb_mse}")

Best Parameters for light g boost: {'learning_rate': 0.1, 'n_estimators': 300, 'num_leaves': 100}
LightGBM - R2: 0.9479277735001643, MAE: 213.41727995651857, MSE: 436889.30295548716


In [5]:
#catboost

import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Assuming you have imported your data as 'data3'

# Separate features and target variable
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.2, random_state=42)


# CatBoost
cat_model = CatBoostRegressor(random_state=42)
cat_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'iterations': [100, 200, 300],
    'depth': [6, 8, 10],
}

cat_grid = GridSearchCV(cat_model, cat_params, scoring='neg_mean_squared_error', cv=3)
cat_grid.fit(x_train, y_train.values.ravel())

# Get the best parameters and model
best_cat_params = cat_grid.best_params_
best_cat_model = cat_grid.best_estimator_
print("Best Parameters forcat boost:", best_cat_params )

# Evaluate models
def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

# Evaluate CatBoost
cat_r2, cat_mae, cat_mse = evaluate_model(best_cat_model, x_test, y_test)
print(f"CatBoost - R2: {cat_r2}, MAE: {cat_mae}, MSE: {cat_mse}")


0:	learn: 2901.2718219	total: 165ms	remaining: 16.3s
1:	learn: 2884.6436959	total: 180ms	remaining: 8.8s
2:	learn: 2868.4445979	total: 192ms	remaining: 6.21s
3:	learn: 2851.6867021	total: 206ms	remaining: 4.94s
4:	learn: 2835.0941215	total: 219ms	remaining: 4.15s
5:	learn: 2818.8136620	total: 236ms	remaining: 3.7s
6:	learn: 2803.5985988	total: 251ms	remaining: 3.34s
7:	learn: 2787.4960778	total: 273ms	remaining: 3.13s
8:	learn: 2771.7631588	total: 293ms	remaining: 2.96s
9:	learn: 2756.6829938	total: 306ms	remaining: 2.75s
10:	learn: 2740.9209560	total: 323ms	remaining: 2.61s
11:	learn: 2726.0915125	total: 339ms	remaining: 2.48s
12:	learn: 2711.7788763	total: 358ms	remaining: 2.39s
13:	learn: 2697.5089770	total: 395ms	remaining: 2.42s
14:	learn: 2683.3717856	total: 423ms	remaining: 2.4s
15:	learn: 2668.9373897	total: 439ms	remaining: 2.3s
16:	learn: 2655.3195122	total: 459ms	remaining: 2.24s
17:	learn: 2641.6515124	total: 484ms	remaining: 2.2s
18:	learn: 2627.5336541	total: 503ms	remain

60:	learn: 2177.8298056	total: 1.89s	remaining: 1.21s
61:	learn: 2169.4772980	total: 1.91s	remaining: 1.17s
62:	learn: 2160.9585420	total: 1.95s	remaining: 1.14s
63:	learn: 2152.9776405	total: 1.98s	remaining: 1.11s
64:	learn: 2145.2719469	total: 1.99s	remaining: 1.07s
65:	learn: 2137.7923135	total: 2.01s	remaining: 1.03s
66:	learn: 2129.9732774	total: 2.02s	remaining: 997ms
67:	learn: 2122.7785929	total: 2.04s	remaining: 958ms
68:	learn: 2115.2445510	total: 2.04s	remaining: 919ms
69:	learn: 2107.4282613	total: 2.05s	remaining: 881ms
70:	learn: 2099.7011604	total: 2.07s	remaining: 845ms
71:	learn: 2092.8525272	total: 2.08s	remaining: 810ms
72:	learn: 2085.4839249	total: 2.11s	remaining: 779ms
73:	learn: 2077.9529380	total: 2.13s	remaining: 747ms
74:	learn: 2070.9041813	total: 2.14s	remaining: 713ms
75:	learn: 2063.8800947	total: 2.15s	remaining: 680ms
76:	learn: 2056.6788538	total: 2.16s	remaining: 646ms
77:	learn: 2050.0504769	total: 2.17s	remaining: 613ms
78:	learn: 2043.1755068	tota

16:	learn: 1687.1859684	total: 293ms	remaining: 1.43s
17:	learn: 1667.3360254	total: 308ms	remaining: 1.4s
18:	learn: 1649.7228341	total: 317ms	remaining: 1.35s
19:	learn: 1632.3537571	total: 326ms	remaining: 1.3s
20:	learn: 1617.9974820	total: 339ms	remaining: 1.27s
21:	learn: 1603.6592466	total: 350ms	remaining: 1.24s
22:	learn: 1590.7639171	total: 362ms	remaining: 1.21s
23:	learn: 1580.7132219	total: 374ms	remaining: 1.18s
24:	learn: 1570.6849505	total: 383ms	remaining: 1.15s
25:	learn: 1560.7899003	total: 392ms	remaining: 1.12s
26:	learn: 1551.3508087	total: 407ms	remaining: 1.1s
27:	learn: 1542.1170726	total: 417ms	remaining: 1.07s
28:	learn: 1533.5637904	total: 428ms	remaining: 1.05s
29:	learn: 1527.6570809	total: 439ms	remaining: 1.02s
30:	learn: 1520.2374605	total: 450ms	remaining: 1s
31:	learn: 1514.9714955	total: 462ms	remaining: 981ms
32:	learn: 1506.7711374	total: 471ms	remaining: 957ms
33:	learn: 1499.0380521	total: 481ms	remaining: 934ms
34:	learn: 1492.6850933	total: 496

79:	learn: 1320.0773246	total: 983ms	remaining: 246ms
80:	learn: 1319.3106212	total: 992ms	remaining: 233ms
81:	learn: 1315.6654503	total: 1s	remaining: 220ms
82:	learn: 1314.1775538	total: 1.01s	remaining: 207ms
83:	learn: 1311.2430738	total: 1.02s	remaining: 195ms
84:	learn: 1309.7867607	total: 1.03s	remaining: 182ms
85:	learn: 1307.9512684	total: 1.05s	remaining: 171ms
86:	learn: 1305.9999831	total: 1.06s	remaining: 159ms
87:	learn: 1303.6819247	total: 1.08s	remaining: 147ms
88:	learn: 1302.2475883	total: 1.09s	remaining: 135ms
89:	learn: 1298.8116114	total: 1.11s	remaining: 123ms
90:	learn: 1294.6348675	total: 1.12s	remaining: 111ms
91:	learn: 1291.4062034	total: 1.13s	remaining: 98.3ms
92:	learn: 1289.3422296	total: 1.14s	remaining: 85.8ms
93:	learn: 1287.0562827	total: 1.15s	remaining: 73.5ms
94:	learn: 1283.6527190	total: 1.16s	remaining: 61.1ms
95:	learn: 1281.7360520	total: 1.18s	remaining: 49.1ms
96:	learn: 1279.1652673	total: 1.19s	remaining: 36.8ms
97:	learn: 1276.2026003	t

38:	learn: 1341.5435955	total: 799ms	remaining: 1.25s
39:	learn: 1336.9169004	total: 825ms	remaining: 1.24s
40:	learn: 1334.6679905	total: 839ms	remaining: 1.21s
41:	learn: 1330.0809810	total: 884ms	remaining: 1.22s
42:	learn: 1325.6150354	total: 897ms	remaining: 1.19s
43:	learn: 1313.5551340	total: 910ms	remaining: 1.16s
44:	learn: 1308.1611500	total: 946ms	remaining: 1.16s
45:	learn: 1300.2534925	total: 966ms	remaining: 1.13s
46:	learn: 1296.1232561	total: 982ms	remaining: 1.11s
47:	learn: 1291.9174223	total: 995ms	remaining: 1.08s
48:	learn: 1289.7405704	total: 1s	remaining: 1.05s
49:	learn: 1284.1695765	total: 1.01s	remaining: 1.01s
50:	learn: 1279.1453786	total: 1.03s	remaining: 992ms
51:	learn: 1275.0670817	total: 1.06s	remaining: 976ms
52:	learn: 1269.9773667	total: 1.08s	remaining: 955ms
53:	learn: 1265.8594368	total: 1.09s	remaining: 931ms
54:	learn: 1263.9231224	total: 1.12s	remaining: 913ms
55:	learn: 1259.5105273	total: 1.13s	remaining: 884ms
56:	learn: 1256.4633331	total: 

...

213:	learn: 479.9925880	total: 9.98s	remaining: 4.01s
214:	learn: 477.6484273	total: 10s	remaining: 3.97s
215:	learn: 476.7463630	total: 10.1s	remaining: 3.92s
216:	learn: 476.2577259	total: 10.1s	remaining: 3.88s
217:	learn: 475.6596581	total: 10.2s	remaining: 3.84s
218:	learn: 474.1310503	total: 10.3s	remaining: 3.8s
219:	learn: 473.4683914	total: 10.3s	remaining: 3.76s
220:	learn: 470.5532522	total: 10.4s	remaining: 3.72s
221:	learn: 468.2647343	total: 10.5s	remaining: 3.67s
222:	learn: 466.3754408	total: 10.5s	remaining: 3.63s
223:	learn: 462.9246118	total: 10.6s	remaining: 3.6s
224:	learn: 461.8574987	total: 10.7s	remaining: 3.56s
225:	learn: 460.9435955	total: 10.8s	remaining: 3.52s
226:	learn: 458.3642301	total: 10.8s	remaining: 3.47s
227:	learn: 457.5845369	total: 10.9s	remaining: 3.43s
228:	learn: 456.1780065	total: 10.9s	remaining: 3.38s
229:	learn: 454.5942749	total: 10.9s	remaining: 3.33s
230:	learn: 453.4109833	total: 11s	remaining: 3.29s
231:	learn: 451.7296236	total: 11.

69:	learn: 818.1646630	total: 2.94s	remaining: 9.66s
70:	learn: 814.6741067	total: 2.99s	remaining: 9.63s
71:	learn: 811.3813758	total: 3.03s	remaining: 9.61s
72:	learn: 807.0665275	total: 3.07s	remaining: 9.56s
73:	learn: 795.8936379	total: 3.12s	remaining: 9.52s
74:	learn: 791.0306172	total: 3.16s	remaining: 9.48s
75:	learn: 788.4044342	total: 3.2s	remaining: 9.44s
76:	learn: 786.1094663	total: 3.24s	remaining: 9.38s
77:	learn: 783.1255535	total: 3.28s	remaining: 9.33s
78:	learn: 780.7587550	total: 3.32s	remaining: 9.28s
79:	learn: 777.7019911	total: 3.36s	remaining: 9.25s
80:	learn: 772.3896880	total: 3.4s	remaining: 9.19s
81:	learn: 768.5484589	total: 3.44s	remaining: 9.14s
82:	learn: 763.2960712	total: 3.48s	remaining: 9.09s
83:	learn: 760.7430544	total: 3.52s	remaining: 9.06s
84:	learn: 757.7853756	total: 3.56s	remaining: 9.01s
85:	learn: 753.0513963	total: 3.6s	remaining: 8.96s
86:	learn: 749.1867265	total: 3.64s	remaining: 8.91s
87:	learn: 744.4797824	total: 3.68s	remaining: 8.

226:	learn: 415.7789174	total: 9.54s	remaining: 3.07s
227:	learn: 414.8898308	total: 9.58s	remaining: 3.02s
228:	learn: 412.6042319	total: 9.62s	remaining: 2.98s
229:	learn: 411.3308266	total: 9.66s	remaining: 2.94s
230:	learn: 410.0575419	total: 9.7s	remaining: 2.9s
231:	learn: 408.0050023	total: 9.75s	remaining: 2.86s
232:	learn: 406.6710384	total: 9.8s	remaining: 2.82s
233:	learn: 405.2567725	total: 9.84s	remaining: 2.77s
234:	learn: 404.6354827	total: 9.88s	remaining: 2.73s
235:	learn: 403.2867146	total: 9.91s	remaining: 2.69s
236:	learn: 402.2757466	total: 9.96s	remaining: 2.65s
237:	learn: 401.1514017	total: 9.99s	remaining: 2.6s
238:	learn: 399.7936732	total: 10s	remaining: 2.56s
239:	learn: 398.5344452	total: 10.1s	remaining: 2.52s
240:	learn: 397.2606508	total: 10.1s	remaining: 2.48s
241:	learn: 396.0739160	total: 10.2s	remaining: 2.43s
242:	learn: 394.6010734	total: 10.2s	remaining: 2.39s
243:	learn: 393.8717210	total: 10.2s	remaining: 2.35s
244:	learn: 391.2746811	total: 10.

83:	learn: 782.4743342	total: 4.24s	remaining: 10.9s
84:	learn: 775.8484272	total: 4.29s	remaining: 10.9s
85:	learn: 773.9906407	total: 4.34s	remaining: 10.8s
86:	learn: 768.9560203	total: 4.38s	remaining: 10.7s
87:	learn: 764.6993468	total: 4.43s	remaining: 10.7s
88:	learn: 760.4455914	total: 4.47s	remaining: 10.6s
89:	learn: 757.2367098	total: 4.53s	remaining: 10.6s
90:	learn: 752.6291378	total: 4.59s	remaining: 10.5s
91:	learn: 749.5747834	total: 4.63s	remaining: 10.5s
92:	learn: 745.2279573	total: 4.7s	remaining: 10.5s
93:	learn: 737.4909946	total: 4.75s	remaining: 10.4s
94:	learn: 735.4577860	total: 4.8s	remaining: 10.4s
95:	learn: 732.5021247	total: 4.84s	remaining: 10.3s
96:	learn: 730.5749765	total: 4.89s	remaining: 10.2s
97:	learn: 728.0157808	total: 4.93s	remaining: 10.2s
98:	learn: 724.4109876	total: 4.98s	remaining: 10.1s
99:	learn: 719.8928018	total: 5.03s	remaining: 10.1s
100:	learn: 718.4050956	total: 5.07s	remaining: 10s
101:	learn: 716.0102377	total: 5.12s	remaining: 9

238:	learn: 458.9745809	total: 11.9s	remaining: 3.03s
239:	learn: 458.1959759	total: 11.9s	remaining: 2.98s
240:	learn: 457.2671316	total: 12s	remaining: 2.93s
241:	learn: 456.5916920	total: 12s	remaining: 2.88s
242:	learn: 456.2004793	total: 12s	remaining: 2.82s
243:	learn: 455.0609428	total: 12.1s	remaining: 2.77s
244:	learn: 453.2154029	total: 12.1s	remaining: 2.72s
245:	learn: 452.6933318	total: 12.2s	remaining: 2.67s
246:	learn: 451.4494342	total: 12.2s	remaining: 2.62s
247:	learn: 449.5455256	total: 12.3s	remaining: 2.57s
248:	learn: 448.9139876	total: 12.3s	remaining: 2.52s
249:	learn: 448.2439024	total: 12.3s	remaining: 2.47s
250:	learn: 445.6397606	total: 12.4s	remaining: 2.42s
251:	learn: 443.8446144	total: 12.4s	remaining: 2.37s
252:	learn: 442.8262666	total: 12.5s	remaining: 2.32s
253:	learn: 441.6651089	total: 12.5s	remaining: 2.27s
254:	learn: 440.2878796	total: 12.6s	remaining: 2.22s
255:	learn: 439.1025852	total: 12.6s	remaining: 2.17s
256:	learn: 437.2254647	total: 12.

In [14]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# XGBoost
xgb_model = xgb.XGBRegressor(random_state=42)
xgb_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
}

xgb_grid = GridSearchCV(xgb_model, xgb_params, scoring='neg_mean_squared_error', cv=3)
xgb_grid.fit(x_train, y_train.values.ravel())

# Get the best parameters and model
best_xgb_params = xgb_grid.best_params_
best_xgb_model = xgb_grid.best_estimator_
print("Best Parameters for xgboost:",best_xgb_params )

# Evaluate XGBoost
xgb_r2, xgb_mae, xgb_mse = evaluate_model(best_xgb_model, x_test, y_test)
print(f"XGBoost - R2: {xgb_r2}, MAE: {xgb_mae}, MSE: {xgb_mse}")


Best Parameters for xgboost: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 300}
XGBoost - R2: 0.9526349660556965, MAE: 205.1279375375704, MSE: 397395.6570582786


In [15]:
joblib.dump(best_xgb_model, 'C:\\Users\\Dell\\OneDrive\\Desktop\\AFs Article\\codes\\Ev\\interface.json')

['C:\\Users\\Dell\\OneDrive\\Desktop\\AFs Article\\codes\\Ev\\interface.json']

In [18]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
}

rf_grid = GridSearchCV(rf_model, rf_params, scoring='neg_mean_squared_error', cv=3)
rf_grid.fit(x_train, y_train.values.ravel())

# Get the best parameters and model
best_rf_params = rf_grid.best_params_
best_rf_model = rf_grid.best_estimator_
print("Best Parameters for rf:",best_rf_params )

# Evaluate Random Forest
rf_r2, rf_mae, rf_mse = evaluate_model(best_rf_model, x_test, y_test)
print(f"Random Forest - R2: {rf_r2}, MAE: {rf_mae}, MSE: {rf_mse}")


Best Parameters for rf: {'max_depth': 20, 'n_estimators': 300}
Random Forest - R2: 0.9486057879702517, MAE: 189.06965692339978, MSE: 431200.71828874183


In [19]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Separate features and target variable
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.2, random_state=42)

# Standardize the data for ANN
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# ANN
ann_model = MLPRegressor(random_state=42)
ann_params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01, 0.1],
}

ann_grid = GridSearchCV(ann_model, ann_params, scoring='neg_mean_squared_error', cv=3)
ann_grid.fit(x_train_scaled, y_train.values.ravel())

# Get the best parameters and model
best_ann_params = ann_grid.best_params_
best_ann_model = ann_grid.best_estimator_
print("Best Parameters for ann:", best_ann_params)

# Evaluate ANN
ann_r2, ann_mae, ann_mse = evaluate_model(best_ann_model, x_test_scaled, y_test)
print(f"ANN - R2: {ann_r2}, MAE: {ann_mae}, MSE: {ann_mse}")


Best Parameters for ann: {'activation': 'tanh', 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01}
ANN - R2: 0.8834636140407386, MAE: 413.27792319079435, MSE: 977747.7141457346


In [20]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Assuming you have imported your data as 'data3'

# Separate features and target variable
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.2, random_state=42)

# KNN
knn_model = KNeighborsRegressor()
knn_params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
}

knn_grid = GridSearchCV(knn_model, knn_params, scoring='neg_mean_squared_error', cv=3)
knn_grid.fit(x_train, y_train.values.ravel())

def evaluate_model(model, x, y_true):
    y_pred = model.predict(x)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    return r2, mae, mse

# Get the best parameters and model
best_knn_params = knn_grid.best_params_
best_knn_model = knn_grid.best_estimator_
print("Best Parameters for KNN:", best_knn_params)

# Evaluate KNN
knn_r2, knn_mae, knn_mse = evaluate_model(best_knn_model, x_test, y_test)
print(f"KNN - R2: {knn_r2}, MAE: {knn_mae}, MSE: {knn_mse}")


Best Parameters for KNN: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
KNN - R2: 0.9406993176563372, MAE: 197.02224261153916, MSE: 497536.50871812314


In [21]:
#extra tree regressor

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Assuming you have imported your data as 'data3'

# Separate features and target variable
x1 = data.iloc[:, :10]
y = data.iloc[:, 11:]

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x1, y, test_size=0.2, random_state=42)

# Extra Trees Regressor
et_model = ExtraTreesRegressor(random_state=42)
et_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

et_grid = GridSearchCV(et_model, et_params, scoring='neg_mean_squared_error', cv=3)
et_grid.fit(x_train, y_train.values.ravel())

# Get the best parameters and model
best_et_params = et_grid.best_params_
best_et_model = et_grid.best_estimator_
print("Best Parameters for Extra Trees:", best_et_params)

# Evaluate Extra Trees
et_r2, et_mae, et_mse = evaluate_model(best_et_model, x_test, y_test)
print(f"Extra Trees - R2: {et_r2}, MAE: {et_mae}, MSE: {et_mse}")


Best Parameters for Extra Trees: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 300}
Extra Trees - R2: 0.9562039593446332, MAE: 150.60391217271462, MSE: 367451.5756339656


In [14]:
joblib.dump(best_lgb_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_lgb_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_lgb_model.json']

In [15]:
joblib.dump(best_cat_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_cat_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_cat_model.json']

In [17]:
joblib.dump(best_xgb_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_xgb_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_xgb_model.json']

In [23]:
joblib.dump(best_rf_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_rf_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_rf_model.json']

In [24]:
joblib.dump(best_ann_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_ann_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_ann_model.json']

In [25]:
joblib.dump(best_knn_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_knn_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_knn_model.json']

In [26]:
joblib.dump(best_et_model,'D:\\my WD\\AFs Article\\codes\\Ev\\best_et_model.json')

['D:\\my WD\\AFs Article\\codes\\Ev\\best_et_model.json']